Test slab_model.py
==================

Jinbo Wang

Jinbo.Wang@jpl.nasa.gov

last update: 9/2/2021

In [ ]:
import slab_model
import pylab as plt
import numpy as np
import xarray as xr
from popy import utils
from scipy.io import loadmat
import pandas as pd


xrod=xr.open_dataset

In [ ]:
fn='../data/from_Hong/PaPa2.mat'
lat0=50.01
lat0=50.01

d=loadmat(fn)

tt=pd.date_range('2007-06-08 04:00:00',periods=8541,freq='1h')

vobs=xr.DataArray(d['VV_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
uobs=xr.DataArray(d['UU_obs'][:].flatten(),dims=('time'),coords={'time':tt} )

#split the signal into low frequency and high frequency
uobs_low,uobs=slab_model.filter_seperation(uobs)
vobs_low,vobs=slab_model.filter_seperation(vobs)

#generate wacm scenario using simple xarray interpolation
wacm_sampling=slab_model.load_wacm_period(lat0)*3600 #convert to seconds

print('The averaged wacm sampling period at lat=%4.1f is %5.2f hours.'%(lat0,wacm_sampling/3600))

tt_wacm=pd.date_range(tt.min(),tt.max(),freq='%is'%wacm_sampling)

#interpolate hourly observations to wacm time to generate synthetic wacm surface velocity
uobs_wacm=uobs.interp(time=tt_wacm)
vobs_wacm=vobs.interp(time=tt_wacm)

#load winds
vwobs=xr.DataArray(d['Vwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )
uwobs=xr.DataArray(d['Uwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )

#generate wacm winds
uw_wacm=uwobs.interp(time=tt_wacm)
vw_wacm=vwobs.interp(time=tt_wacm)

#select a smaller time window for analysis
t0,t1='2008-02-10','2008-02-19'

u=uobs_wacm.sel(time=slice(t0,t1))
v=vobs_wacm.sel(time=slice(t0,t1))
t_uv=u.time.values

uw=uw_wacm.sel(time=slice(t0,t1))
vw=vw_wacm.sel(time=slice(t0,t1))
t_tau=uw.time.values

vobs_sub=vobs.sel(time=slice(t_uv.min(),t_uv.max()))
uobs_sub=uobs.sel(time=slice(t_uv.min(),t_uv.max()))
t_output=vobs_sub.time.values

f0=utils.lat2f(lat0)
print('The inertial period = %5.2f hours'%(2*np.pi/f0/86400*24))


u_pred,v_pred=slab_model.optimize_slab_noshear_withtide(t_uv,u.data,v.data,t_tau,uw.data,vw.data,f0,t_output,has_tides=False)

error=(((u_pred-uobs_sub.values)**2+(v_pred-vobs_sub.values)**2).mean()/2)**0.5
print('The prediction error for the current speed is %6.3f cm'%(error*100))

fig,ax=plt.subplots(3,1,figsize=(10,10),sharex=True)
ax[0].plot(t_output,u_pred,label='u pred')
ax[0].plot(t_uv,u,'o',label='u obs')
ax[0].plot(t_output,uobs.interp(time=t_output),label='truth')

ax[0].legend()

ax[1].plot(t_output,v_pred,label='v pred')
ax[1].plot(t_uv,v,'o',label='v obs')
ax[1].plot(t_output,vobs.interp(time=t_output),label='truth')
ax[1].legend()


ax[2].plot(t_tau,uw,label='u wind')
ax[2].plot(t_tau,vw,label='v wind')
ax[2].legend()

plt.tight_layout()

Test the fitting_error in slab_model. The fitting_error subroutine simplifies the error calculation. 

In [ ]:
fn='../data/from_Hong/PaPa2.mat'
lat0=50.01

d=loadmat(fn)

tt=pd.date_range('2007-06-08 04:00:00',periods=8541,freq='1h')

#load velocity and create xarrays 
vobs=xr.DataArray(d['VV_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
uobs=xr.DataArray(d['UU_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
print(vobs)
#split the signal into low frequency and high frequency
uobs_low,uobs=slab_model.filter_seperation(uobs)
vobs_low,vobs=slab_model.filter_seperation(vobs)

#load winds and create xarrays
vwobs=xr.DataArray(d['Vwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )
uwobs=xr.DataArray(d['Uwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )


#t0,t1 can be looped to analyse a long time series 

t0,t1='2008-02-10','2008-02-19'
error,truth=slab_model.fitting_error(uobs,vobs,uwobs,vwobs,t0,t1,lat0,wacm_error_v=0,wacm_error_wind=0)
print(error)
print('The fitting error based on the time window',t0,t1, 
      'is %6.3f cm/s. Truth=%6.3f. Error variance ratio is %6.3f.'%(error*100,truth*100,(error/truth)**2))

#with measurement errors
error,truth=slab_model.fitting_error(uobs,vobs,uwobs,vwobs,t0,t1,lat0,wacm_error_v=0.1,wacm_error_wind=2)
print('With WaCM measurement errors, the fitting error based on the time window',
      t0,t1, 'is %6.3f cm/s. Truth=%6.3f. Error variance ratio is %6.3f.'%(error*100,truth*100,(error/truth)**2))

Test looping a whole year
=======================

In [ ]:
fn='../data/from_Hong/PaPa2.mat'
lat0=50.01

d=loadmat(fn)

tt=pd.date_range('2007-06-08 04:00:00',periods=8541,freq='1h')

#load velocity and create xarrays 
vobs=xr.DataArray(d['VV_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
uobs=xr.DataArray(d['UU_obs'][:].flatten(),dims=('time'),coords={'time':tt} )

#split the signal into low frequency and high frequency
uobs_low,uobs=slab_model.filter_seperation(uobs)
vobs_low,vobs=slab_model.filter_seperation(vobs)

#load winds and create xarrays
vwobs=xr.DataArray(d['Vwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )
uwobs=xr.DataArray(d['Uwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )



errors=[]
truths=[]


t0s=pd.date_range(vobs.time[0].values,vobs.time[-1].values,freq='10D')

for t0 in t0s:
    
    error,truth=slab_model.fitting_error(uobs,vobs,uwobs,vwobs,t0,t0+np.timedelta64(20,'D'),lat0,wacm_error_v=0,wacm_error_wind=0,has_tides=True)
    #print(error)
    #print('The fitting error based on the time window',t0,t1, 
    #  'is %6.3f cm/s. Truth=%6.3f. Error variance ratio is %6.3f.'%(error*100,truth*100,(error/truth)**2))
    errors.append(error)
    truths.append(truth)
    print(t0,error,truth)
    del error, truth

Test long-time series reconstruction
==================================

The main routine is 

     u_pred,v_pred=slab_model.reconstruct_NIO(uobs,vobs,uwobs,vwobs,lat0,
                                             fitting_window=5,
                                             t_out=None,
                                             wacm_error_v=0.1,
                                             wacm_error_wind=4.0,
                                             has_tides=False,
                                             is_windstress=False)
                                  

* Make sure the input velocities (uobs, vobs) and winds (uwobs, vwobs) are xarray with 'time' axis
* fitting_window is the time window in days for fitting the NIO, 5-10-day seems to be a good value for mid-latitude, i.e., 12-20 data wacm observations in the optimization
* t_out is the time axis on which the output is interpolated

The procedure is as follows

1. Take high frequency observations, remove tides
1. bandpass to get the filtered NIO, save for later comparison
1. split the signal into low-frequency and high-frequency, assume low-frequency is known through altimetry
1. take high-pass filtered data, plug in to reconstruct_NIO, get prediction

Caveat: the WaCM data was taken after the highpass and removing tides.


In [ ]:

import slab_model
import pylab as plt
import numpy as np
import xarray as xr
from popy import utils
from scipy.io import loadmat
import pandas as pd

xrod=xr.open_dataset

#load data station papa
fn='../data/from_Hong/PaPa2.mat'
lat0=50.01
tt=pd.date_range('2007-06-08 04:00:00',periods=8541,freq='1h')
t0,t1='2007-09-22','2007-10-02'


#fn='../data/from_Hong/KEO2.mat'
#lat0=32.3
#tt=pd.date_range('2007-09-26 11:00:00',periods=8255,freq='1h')
#t0,t1='2007-11-20','2007-12-20'
#t0,t1='2007-12-13','2008-01-30'

d=loadmat(fn)

#load velocity and create xarrays 
vobs=xr.DataArray(d['VV_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
uobs=xr.DataArray(d['UU_obs'][:].flatten(),dims=('time'),coords={'time':tt} )

#load winds and create xarrays
vwobs=xr.DataArray(d['Vwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )
uwobs=xr.DataArray(d['Uwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )#.sel(time=slice(t0,t1))


#split the signal into low frequency and high frequency
uobs_low0,uobs=slab_model.filter_seperation(uobs,cutoff=1/30)
vobs_low0,vobs=slab_model.filter_seperation(vobs,cutoff=1/30)

uobs,vobs=slab_model.remove_tides([uobs,vobs],lat0)
#vobs,v_tide=slab_model.remove_tides(vobs,lat0)

uobs_nio=slab_model.NIO_bandpass(uobs,lat0).sel(time=slice(t0,t1))
#split the signal into low frequency and high frequency
#uobs_low,uobs=slab_model.filter_seperation(uobs,cutoff=1/4)
#vobs_low,vobs=slab_model.filter_seperation(vobs,cutoff=1/4)

orbit_version='700_1800'
orbit_version='500_1000'


uwacm=slab_model.interp_wacm(uobs,lat0,orbit_version)
vwacm=slab_model.interp_wacm(vobs,lat0,orbit_version)
uwacm_low,uwacm=slab_model.filter_seperation(uwacm,cutoff=1/5)
vwacm_low,vwacm=slab_model.filter_seperation(vwacm,cutoff=1/5)

uwacm=uwacm.sel(time=slice(t0,t1))
vwacm=vwacm.sel(time=slice(t0,t1))



uw_wacm=slab_model.interp_wacm(uwobs,lat0,orbit_version).sel(time=slice(t0,t1))
vw_wacm=slab_model.interp_wacm(vwobs,lat0,orbit_version).sel(time=slice(t0,t1))

uobs=uobs.sel(time=slice(t0,t1))
vobs=vobs.sel(time=slice(t0,t1))



u=0;v=0

#loop n_times for ensemble, each time has different noise specified by random error with std of wacm_error_v (m/s), wacm_error_wind (m/s)
n_times=1
input_wacm=True

for i in range(n_times):
    if input_wacm:
        u_pred,v_pred=slab_model.reconstruct_NIO(uwacm,vwacm,uw_wacm,vw_wacm,lat0,
                                             fitting_window=10,
                                             t_out=None,
                                             wacm_error_v=0.5,
                                             wacm_error_wind=[0,0],
                                             has_tides=False,
                                             is_windstress=False,
                                             orbit_version=orbit_version,
                                             input_wacm_data=True)
    else:
        u_pred,v_pred=slab_model.reconstruct_NIO(uobs,vobs,uw_wacm,vw_wacm,lat0,
                                             fitting_window=10,
                                             t_out=None,
                                             wacm_error_v=0.0,
                                             wacm_error_wind=[0,0],
                                             has_tides=False,
                                             is_windstress=False,
                                            orbit_version=orbit_version)
    u+=u_pred
    v+=v_pred
    #plt.plot(u_pred.time,u_pred.values,'.')

#reconstruct the whole time series
fig=plt.figure(figsize=(10,4))
uobs.plot(lw=2,color='gray',label='Truth')

#(uwacm_low+uwacm+np.random.normal(0,0.1,uwacm.size)).plot(color='b',marker='o',label='WaCM')
(uwacm_low+uwacm).plot(color='b',marker='o',label='WaCM')

u=u/n_times;v=v/n_times
u=u-u.mean()
u.plot(color='r',label='Predicted based on WaCM obs')
uobs_nio.plot(color='g',label='Filtered NIO truth')
plt.legend(fontsize=10)
plt.xlim(np.datetime64(t0),np.datetime64(t0)+np.timedelta64(10,'D'))
plt.xlabel('Time',fontsize=16)
plt.ylabel('U velocity (m/s)',fontsize=16)
plt.ylim(-0.4,0.4)
plt.grid(True)
plt.tight_layout()


plt.savefig('figures/nio_reconstruction_demo_papa_%s.pdf'%orbit_version)
plt.savefig('figures/nio_reconstruction_demo_papa_%s.png'%orbit_version,dpi=300)



In [ ]:
f0=utils.lat2f(lat0)
up,vp=slab_model.optimize_slab_noshear_withtide(uwacm.time.values,uwacm,vwacm,
                                                uw_wacm.time.values,uw_wacm,vw_wacm,
                                                f0,t_out=None,
                                   has_tides=False,
                                   T_tide=[1.07581, 0.99727, 0.517525, 0.5, 0.2587625, 0.2587625],
                                   is_windstress=False)

# Test using higher frequency winds without being on the WaCM grid

In [ ]:

import slab_model
import pylab as plt
import numpy as np
import xarray as xr
from popy import utils
from scipy.io import loadmat
import pandas as pd

xrod=xr.open_dataset

#load data station papa
fn='../data/from_Hong/PaPa2.mat'
lat0=50.01
tt=pd.date_range('2007-06-08 04:00:00',periods=8541,freq='1h')
t0,t1='2007-09-22','2007-10-02'


#fn='../data/from_Hong/KEO2.mat'
#lat0=32.3
#tt=pd.date_range('2007-09-26 11:00:00',periods=8255,freq='1h')
#t0,t1='2007-11-20','2007-12-20'
#t0,t1='2007-12-13','2008-01-30'

d=loadmat(fn)

#load velocity and create xarrays 
vobs=xr.DataArray(d['VV_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
uobs=xr.DataArray(d['UU_obs'][:].flatten(),dims=('time'),coords={'time':tt} )

#load winds and create xarrays
vwobs=xr.DataArray(d['Vwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )
uwobs=xr.DataArray(d['Uwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )


uwacm=slab_model.interp_wacm(uobs,lat0,orbit_version)
vwacm=slab_model.interp_wacm(vobs,lat0,orbit_version)

fig=plt.figure(3,1,figsize=(10,10))
uobs.sel(time=slice(t0,t1)).plot(ax=ax[0])
uwacm.sel(time=slice(t0,t1)).plot(ax=ax[0],marker='o')

#split the signal into low frequency and high frequency
uobs_low0,uobs=slab_model.filter_seperation(uobs,cutoff=1/30)
vobs_low0,vobs=slab_model.filter_seperation(vobs,cutoff=1/30)

uobs,vobs=slab_model.remove_tides([uobs,vobs],lat0)
#vobs,v_tide=slab_model.remove_tides(vobs,lat0)

uobs_nio=slab_model.NIO_bandpass(uobs,lat0).sel(time=slice(t0,t1))
#split the signal into low frequency and high frequency
#uobs_low,uobs=slab_model.filter_seperation(uobs,cutoff=1/4)
#vobs_low,vobs=slab_model.filter_seperation(vobs,cutoff=1/4)

orbit_version='700_1800'
orbit_version='500_1000'


uwacm_low,uwacm=slab_model.filter_seperation(uwacm,cutoff=1/5)
vwacm_low,vwacm=slab_model.filter_seperation(vwacm,cutoff=1/5)

uwacm=uwacm.sel(time=slice(t0,t1))
vwacm=vwacm.sel(time=slice(t0,t1))


uw_wacm=uwobs.sel(time=slice(t0,t1))
vw_wacm=vwobs.sel(time=slice(t0,t1))

uobs=uobs.sel(time=slice(t0,t1))
vobs=vobs.sel(time=slice(t0,t1))

u=0;v=0

#loop n_times for ensemble, each time has different noise specified by random error with std of wacm_error_v (m/s), wacm_error_wind (m/s)
n_times=1
input_wacm=True

for i in range(n_times):
    if input_wacm:
        u_pred,v_pred=slab_model.reconstruct_NIO(uwacm,vwacm,uwobs,vwobs,lat0,
                                             fitting_window=10,
                                             t_out=None,
                                             wacm_error_v=0.5,
                                             wacm_error_wind=[0,0],
                                             has_tides=False,
                                             is_windstress=False,
                                             orbit_version=orbit_version,
                                             input_wacm_data=True)
    else:
        u_pred,v_pred=slab_model.reconstruct_NIO(uobs,vobs,uw_wacm,vw_wacm,lat0,
                                             fitting_window=10,
                                             t_out=None,
                                             wacm_error_v=0.0,
                                             wacm_error_wind=[0,0],
                                             has_tides=False,
                                             is_windstress=False,
                                            orbit_version=orbit_version)
    u+=u_pred
    v+=v_pred
    #plt.plot(u_pred.time,u_pred.values,'.')

#reconstruct the whole time series
uobs.plot(lw=2,color='gray',label='Truth')

#(uwacm_low+uwacm+np.random.normal(0,0.1,uwacm.size)).plot(color='b',marker='o',label='WaCM')
(uwacm_low+uwacm).plot(color='b',marker='o',label='WaCM')

u=u/n_times;v=v/n_times
#u=u-u.mean()
u.plot(color='r',label='Predicted based on WaCM obs')
uobs_nio.plot(color='g',label='Filtered NIO truth')
plt.legend(fontsize=10)
plt.xlim(np.datetime64(t0),np.datetime64(t0)+np.timedelta64(10,'D'))
plt.xlabel('Time',fontsize=16)
plt.ylabel('U velocity (m/s)',fontsize=16)
plt.ylim(-0.4,0.4)
plt.grid(True)
plt.tight_layout()


# WaCM was generated from the original

Highpass filtering and detides will be performed on the simulated WaCM data.

In [ ]:
import slab_model
import pylab as plt
import numpy as np
import xarray as xr
from popy import utils
from scipy.io import loadmat
import pandas as pd


xrod=xr.open_dataset

#load data station papa
fn='../data/from_Hong/PaPa2.mat'
lat0=50.01
tt=pd.date_range('2007-06-08 04:00:00',periods=8541,freq='1h')
t0,t1='2007-08-20','2007-08-29'


#fn='../data/from_Hong/KEO2.mat'
#lat0=32.3
#tt=pd.date_range('2007-09-26 11:00:00',periods=8255,freq='1h')
#t0,t1='2007-11-20','2007-12-20'
#t0,t1='2007-12-13','2008-01-30'

d=loadmat(fn)

#load velocity and create xarrays 
vobs=xr.DataArray(d['VV_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
uobs=xr.DataArray(d['UU_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
#load winds and create xarrays
vwobs=xr.DataArray(d['Vwind_o'][:].flatten(),dims=('time'),coords={'time':tt} ).sel(time=slice(t0,t1))
uwobs=xr.DataArray(d['Uwind_o'][:].flatten(),dims=('time'),coords={'time':tt} ).sel(time=slice(t0,t1))

#split the true signal into low frequency and high frequency
uobs_low0,uobs=slab_model.filter_seperation(uobs,cutoff=1/30)
vobs_low0,vobs=slab_model.filter_seperation(vobs,cutoff=1/30)



wacm_error_v=0.05 #5cm/s noise
#average pixel for 5km
N_average=6 #6 pixels for 30km
wacm_error_v=wacm_error_v/N_average

wacm_error_wind=[0,0] #first number is percentage in speed, second is for the direction in degrees (both are in std statistics)

orbit_version='700_1800'

wacm_u,wacm_v,wacm_uw,wacm_vw=slab_model.generate_wacm(uobs,vobs,uwobs,vwobs,lat0,
                                                wacm_error_v=wacm_error_v,
                                                wacm_error_wind=wacm_error_wind,
                                               orbit_version=orbit_version)


fig,ax=plt.subplots(3,1,figsize=(20,10))

utmo=(uobs_low0+uobs).sel(time=slice(t0,t1))
utmw=(uobs_low0.interp(time=wacm_u.time)+wacm_u).sel(time=slice(t0,t1))

ax[0].plot(utmo.time,utmo,label='original')
ax[0].plot(utmw.time,utmw,'-o',label='WaCM')

ax[0].legend()

def get_highpass(data,lat0):
    low,high=slab_model.filter_seperation(wacm_u,cutoff=1/5)
    high,tide=slab_model.remove_tides(high,lat0)
    return low+tide,high

wacm_u_low,wacm_u_high=get_highpass(wacm_u,lat0)
wacm_v_low,wacm_v_high=get_highpass(wacm_v,lat0)

utmo=wacm_u.sel(time=slice(t0,t1))
utmw=wacm_u_high.sel(time=slice(t0,t1))

ax[1].plot(utmo.time,utmo,label='wacm')
ax[1].plot(utmw.time,utmw,'o',label='WaCM high')

ax[1].legend()



uobs,u_tide=slab_model.remove_tides(uobs,lat0)
vobs,v_tide=slab_model.remove_tides(vobs,lat0)

uobs_nio=slab_model.NIO_bandpass(uobs,lat0).sel(time=slice(t0,t1))

uobs=uobs.sel(time=slice(t0,t1))
vobs=vobs.sel(time=slice(t0,t1))

#split the signal into low frequency and high frequency
uobs_low,uobs=get_highpass(uobs,lat0)
vobs_low,vobs=get_highpass(vobs,lat0)



#loop n_times for ensemble, each time has different noise specified by random error with std of wacm_error_v (m/s), wacm_error_wind (m/s)
n_times=1
for i in range(n_times):
    #u_pred,v_pred=slab_model.reconstruct_NIO(wacm_u_high.sel(time=slice(t0,t1)),
    #                                         wacm_v_high.sel(time=slice(t0,t1)),
    #                                         wacm_uw.sel(time=slice(t0,t1)),
    #                                         wacm_vw.sel(time=slice(t0,t1)),
    u_pred,v_pred=slab_model.reconstruct_NIO(uobs.sel(time=slice(t0,t1)),
                                             vobs.sel(time=slice(t0,t1)),
                                             wacm_uw.sel(time=slice(t0,t1)),
                                             wacm_vw.sel(time=slice(t0,t1)),
                                             lat0,
                                             fitting_window=10,
                                             t_out=None,
                                             wacm_error_v=0.,
                                             wacm_error_wind=[0,0],
                                             has_tides=False,
                                             is_windstress=False,
                                             orbit_version='700_1800',
                                             input_wacm_data=False)
    u+=u_pred
    v+=v_pred
    #plt.plot(u_pred.time,u_pred.values,'.')
u=u/n_times;v=v/n_times
u.plot(ax=ax[2],color='k',label='Predicted based on WaCM obs')
uobs_nio.plot(ax=ax[2],label='Filtered NIO truth')
plt.legend()

#reconstruct the whole time series
fig=plt.figure(figsize=(10,4))
uobs.plot(label='observations')
uwacm=slab_model.interp_wacm(uobs,lat0)
plt.plot(uwacm.time,uwacm.values,'bo')
u=0;v=0

In [ ]:
fig,ax=plt.subplots(4,1,figsize=(30,10))
#uobs.plot(label='observations')
uobs_nio.plot(ax=ax[0],label='observations NIO')
u.plot(ax=ax[0],label='predicted')
ax[0].plot(uwacm.time,uwacm.values,'bo')

uobs_nio.plot(ax=ax[1],label='observations NIO')
u.plot(ax=ax[1],label='predicted')

uwobs.plot(ax=ax[2])
vwobs.plot(ax=ax[3])

print((u-uobs_nio).std(),uobs_nio.std())

plt.legend()
plt.tight_layout()